## Pool

pool 是catboost中的数据处理函数，主要有三个重要的参数。
 - data：数据
 - label：标签
 - cat_features：列是否为分类数据，对分类数据进行处理是catboost最重要的一个属性。
 
除此之外，还有几个参数可用，如下：
 - delimiter：分隔符，默认分隔符为'\t’
 - has_header：是否读取第一列为列名。
 - thread_count：读取数据时候用的线程数
以上三个参数在读取文件时比较重要，依靠他们我们可以快速的使用pool从文件中读取数据，其他参数如设置权重等，也是可行的，但是本处不再介绍，请自行查询。



## 例子
### 下面是官网自带的一个例子

In [1]:
import numpy as np
from catboost import CatBoostClassifier, FeaturesData, Pool
# Initialize data
train_data = Pool(
    data=FeaturesData(
        num_feature_data=np.array([[1, 4, 5, 6], [4, 5, 6, 7], [30, 40, 50, 60]], dtype=np.float32),
        cat_feature_data=np.array([[b"a", b"b"], [b"a", b"b"], [b"c", b"d"]], dtype=object)
    ),
    label=[1, 1, -1]
)
test_data = Pool(
    data=FeaturesData(
        num_feature_data=np.array([[2, 4, 6, 8], [1, 4, 50, 60]], dtype=np.float32),
        cat_feature_data=np.array([[b"a", b"b"], [b"a", b"d"]], dtype=object)
    )
)
# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=2, learning_rate=1, depth=2, loss_function='Logloss')
# Fit model
model.fit(train_data)
# Get predicted classes
preds_class = model.predict(test_data)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(test_data)
# Get predicted RawFormulaVal
preds_raw = model.predict(test_data, prediction_type='RawFormulaVal')

0:	learn: 0.2806911	total: 53.4ms	remaining: 53.4ms
1:	learn: 0.1282636	total: 54.5ms	remaining: 0us


In [24]:
preds_class

array([ 0.488     , -0.33007812])

这个例子很好的展示了pool的用法，但是对于我们而言是不足够的。下面的这个例子展示了使用pool从文件中读取UCI红酒质量数据集和从numpy数据格式中读取两种方案。

### 使用pool直接从numpy中读取数据

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd

def GetNewDataByPandas():
    wine = pd.read_csv("/home/fonttian/Data/UCI/wine/wine.csv")
    wine['alcohol**2'] = pow(wine["alcohol"], 2)
    wine['volatileAcidity*alcohol'] = wine["alcohol"] * wine['volatile acidity']
    y = np.array(wine.quality)
    X = np.array(wine.drop("quality", axis=1))
    # X = np.array(wine)

    columns = np.array(wine.columns)

    return X, y, columns

In [4]:
# Read wine quality data from file
X, y, wineNames = GetNewDataByPandas()

# split data to [[0.8,0.2],01]
x_train, x_predict, y_train, y_predict = train_test_split(X, y, test_size=0.10, random_state=100)

In [8]:
from catboost import CatBoostRegressor, FeaturesData, Pool

train_pool = Pool(data=x_train,label=y_train)
test_pool = Pool(data=x_predict,label=y_predict)

In [29]:
# Initialize CatBoostClassifier
model = CatBoostRegressor(n_estimators=1000,learning_rate=1,depth=7, loss_function='RMSE',random_seed=100,early_stopping_rounds = 100)
# Fit model
model.fit(train_pool)
# Get predicted result
prediction = model.predict(test_pool)
prediction

0:	learn: 0.8521521	total: 31.4ms	remaining: 31.4s
1:	learn: 0.7601005	total: 47.2ms	remaining: 23.6s
2:	learn: 0.7227098	total: 60.5ms	remaining: 20.1s
3:	learn: 0.7165734	total: 70ms	remaining: 17.4s
4:	learn: 0.6985662	total: 79.5ms	remaining: 15.8s
5:	learn: 0.6909038	total: 88.7ms	remaining: 14.7s
6:	learn: 0.6890470	total: 97.6ms	remaining: 13.8s
7:	learn: 0.6771394	total: 107ms	remaining: 13.2s
8:	learn: 0.6741907	total: 115ms	remaining: 12.7s
9:	learn: 0.6723554	total: 125ms	remaining: 12.4s
10:	learn: 0.6719224	total: 134ms	remaining: 12.1s
11:	learn: 0.6707629	total: 144ms	remaining: 11.9s
12:	learn: 0.6705063	total: 153ms	remaining: 11.6s
13:	learn: 0.6667419	total: 162ms	remaining: 11.4s
14:	learn: 0.6588306	total: 171ms	remaining: 11.2s
15:	learn: 0.6579094	total: 180ms	remaining: 11.1s
16:	learn: 0.6503348	total: 189ms	remaining: 10.9s
17:	learn: 0.6449906	total: 199ms	remaining: 10.8s
18:	learn: 0.6416760	total: 207ms	remaining: 10.7s
19:	learn: 0.6297023	total: 216ms	re

161:	learn: 0.4883500	total: 1.86s	remaining: 9.61s
162:	learn: 0.4883230	total: 1.87s	remaining: 9.61s
163:	learn: 0.4881948	total: 1.89s	remaining: 9.62s
164:	learn: 0.4881056	total: 1.9s	remaining: 9.6s
165:	learn: 0.4880053	total: 1.91s	remaining: 9.57s
166:	learn: 0.4880033	total: 1.92s	remaining: 9.56s
167:	learn: 0.4875235	total: 1.93s	remaining: 9.54s
168:	learn: 0.4869897	total: 1.94s	remaining: 9.52s
169:	learn: 0.4868520	total: 1.95s	remaining: 9.5s
170:	learn: 0.4865809	total: 1.95s	remaining: 9.48s
171:	learn: 0.4865620	total: 1.96s	remaining: 9.46s
172:	learn: 0.4865615	total: 1.97s	remaining: 9.4s
173:	learn: 0.4865518	total: 1.98s	remaining: 9.38s
174:	learn: 0.4865164	total: 1.99s	remaining: 9.37s
175:	learn: 0.4864647	total: 2s	remaining: 9.35s
176:	learn: 0.4864284	total: 2s	remaining: 9.33s
177:	learn: 0.4848807	total: 2.01s	remaining: 9.3s
178:	learn: 0.4845570	total: 2.02s	remaining: 9.28s
179:	learn: 0.4843851	total: 2.03s	remaining: 9.26s
180:	learn: 0.4838961	t

326:	learn: 0.4259807	total: 3.49s	remaining: 7.18s
327:	learn: 0.4259776	total: 3.5s	remaining: 7.17s
328:	learn: 0.4256190	total: 3.52s	remaining: 7.17s
329:	learn: 0.4255776	total: 3.53s	remaining: 7.16s
330:	learn: 0.4255675	total: 3.54s	remaining: 7.15s
331:	learn: 0.4255247	total: 3.54s	remaining: 7.13s
332:	learn: 0.4254989	total: 3.55s	remaining: 7.12s
333:	learn: 0.4254879	total: 3.56s	remaining: 7.11s
334:	learn: 0.4151186	total: 3.57s	remaining: 7.09s
335:	learn: 0.4145112	total: 3.58s	remaining: 7.08s
336:	learn: 0.4141098	total: 3.59s	remaining: 7.07s
337:	learn: 0.4140435	total: 3.6s	remaining: 7.05s
338:	learn: 0.4139127	total: 3.61s	remaining: 7.04s
339:	learn: 0.4137743	total: 3.62s	remaining: 7.03s
340:	learn: 0.4137422	total: 3.63s	remaining: 7.02s
341:	learn: 0.4131630	total: 3.64s	remaining: 7.01s
342:	learn: 0.4119145	total: 3.65s	remaining: 7s
343:	learn: 0.4117587	total: 3.67s	remaining: 7s
344:	learn: 0.4117149	total: 3.68s	remaining: 6.99s
345:	learn: 0.411691

497:	learn: 0.3720334	total: 5.68s	remaining: 5.73s
498:	learn: 0.3712234	total: 5.7s	remaining: 5.72s
499:	learn: 0.3711438	total: 5.72s	remaining: 5.72s
500:	learn: 0.3710798	total: 5.73s	remaining: 5.71s
501:	learn: 0.3709280	total: 5.75s	remaining: 5.7s
502:	learn: 0.3709115	total: 5.76s	remaining: 5.69s
503:	learn: 0.3707954	total: 5.77s	remaining: 5.68s
504:	learn: 0.3707833	total: 5.78s	remaining: 5.67s
505:	learn: 0.3707735	total: 5.79s	remaining: 5.65s
506:	learn: 0.3707620	total: 5.8s	remaining: 5.64s
507:	learn: 0.3706494	total: 5.81s	remaining: 5.63s
508:	learn: 0.3705935	total: 5.83s	remaining: 5.62s
509:	learn: 0.3705086	total: 5.84s	remaining: 5.61s
510:	learn: 0.3704298	total: 5.85s	remaining: 5.6s
511:	learn: 0.3703432	total: 5.86s	remaining: 5.59s
512:	learn: 0.3703073	total: 5.88s	remaining: 5.58s
513:	learn: 0.3702032	total: 5.89s	remaining: 5.57s
514:	learn: 0.3699243	total: 5.9s	remaining: 5.56s
515:	learn: 0.3698882	total: 5.92s	remaining: 5.55s
516:	learn: 0.369

656:	learn: 0.3291837	total: 7.48s	remaining: 3.9s
657:	learn: 0.3290859	total: 7.49s	remaining: 3.89s
658:	learn: 0.3289559	total: 7.5s	remaining: 3.88s
659:	learn: 0.3287361	total: 7.52s	remaining: 3.87s
660:	learn: 0.3286568	total: 7.53s	remaining: 3.86s
661:	learn: 0.3285978	total: 7.54s	remaining: 3.85s
662:	learn: 0.3285353	total: 7.55s	remaining: 3.84s
663:	learn: 0.3284976	total: 7.56s	remaining: 3.82s
664:	learn: 0.3284936	total: 7.57s	remaining: 3.81s
665:	learn: 0.3284560	total: 7.58s	remaining: 3.8s
666:	learn: 0.3284293	total: 7.59s	remaining: 3.79s
667:	learn: 0.3284257	total: 7.59s	remaining: 3.77s
668:	learn: 0.3284117	total: 7.61s	remaining: 3.76s
669:	learn: 0.3284106	total: 7.62s	remaining: 3.75s
670:	learn: 0.3283902	total: 7.63s	remaining: 3.74s
671:	learn: 0.3283761	total: 7.64s	remaining: 3.73s
672:	learn: 0.3283236	total: 7.65s	remaining: 3.72s
673:	learn: 0.3283018	total: 7.66s	remaining: 3.7s
674:	learn: 0.3282335	total: 7.67s	remaining: 3.69s
675:	learn: 0.32

827:	learn: 0.3013833	total: 9.25s	remaining: 1.92s
828:	learn: 0.3013629	total: 9.26s	remaining: 1.91s
829:	learn: 0.3012607	total: 9.27s	remaining: 1.9s
830:	learn: 0.3008849	total: 9.28s	remaining: 1.89s
831:	learn: 0.3008409	total: 9.29s	remaining: 1.88s
832:	learn: 0.3008354	total: 9.3s	remaining: 1.86s
833:	learn: 0.3007285	total: 9.31s	remaining: 1.85s
834:	learn: 0.3007231	total: 9.32s	remaining: 1.84s
835:	learn: 0.3007020	total: 9.34s	remaining: 1.83s
836:	learn: 0.3001527	total: 9.35s	remaining: 1.82s
837:	learn: 0.3000729	total: 9.36s	remaining: 1.81s
838:	learn: 0.2995496	total: 9.37s	remaining: 1.8s
839:	learn: 0.2995044	total: 9.38s	remaining: 1.79s
840:	learn: 0.2994106	total: 9.39s	remaining: 1.77s
841:	learn: 0.2993779	total: 9.4s	remaining: 1.76s
842:	learn: 0.2993734	total: 9.41s	remaining: 1.75s
843:	learn: 0.2992199	total: 9.42s	remaining: 1.74s
844:	learn: 0.2991224	total: 9.43s	remaining: 1.73s
845:	learn: 0.2988814	total: 9.44s	remaining: 1.72s
846:	learn: 0.29

993:	learn: 0.2828756	total: 11s	remaining: 66.7ms
994:	learn: 0.2828386	total: 11.1s	remaining: 55.6ms
995:	learn: 0.2827749	total: 11.1s	remaining: 44.5ms
996:	learn: 0.2826461	total: 11.1s	remaining: 33.4ms
997:	learn: 0.2815564	total: 11.1s	remaining: 22.2ms
998:	learn: 0.2813901	total: 11.1s	remaining: 11.1ms
999:	learn: 0.2813008	total: 11.1s	remaining: 0us


array([6.25112369, 5.17667817, 5.16019615, 5.8821045 , 6.38472155,
       7.1276523 , 6.06581634, 6.15995758, 5.62439085, 7.0583987 ,
       6.79771803, 5.28261571, 5.33692304, 5.07560517, 5.78262659,
       4.21664252, 5.04530613, 4.81145771, 5.14257582, 5.49505728,
       4.8576215 , 5.11367475, 5.81303022, 7.30126798, 6.21893673,
       6.58312355, 4.84858669, 5.61402198, 5.51835405, 5.68727183,
       5.28061329, 4.8824021 , 7.02848709, 5.8965777 , 5.5359377 ,
       6.24415065, 5.73756608, 5.21804026, 4.9150496 , 5.20642933,
       5.44050438, 6.21223741, 5.47210198, 5.7802992 , 5.54194277,
       5.61513984, 5.33440727, 5.09829063, 5.80682656, 5.98734305,
       5.57335949, 5.30407525, 5.4549325 , 5.12086546, 6.52176357,
       6.33467324, 6.6197769 , 5.43452896, 5.07098266, 4.72927537,
       6.25673496, 5.40181057, 5.49927577, 6.8938378 , 5.59840253,
       5.36523151, 5.42838296, 5.37638335, 5.05026422, 5.44565727,
       6.39386654, 5.93899099, 5.30799687, 7.00206975, 7.78553

In [31]:
from sklearn.metrics import mean_squared_error
print("RMSE of the example : %.3f "%np.sqrt(mean_squared_error(y_predict, prediction)))

RMSE of the example : 0.692 


### 直接从文件中读取数据
#### 使用numpy 存储为txt文件
我们这里直接使用`numpy.savetxt()`存储为csv文件,存储时候分隔符也使用逗号。这是pandas默认的csv文件存储格式，当然其他格式都是可以的。这里只是一个用法示范。

In [49]:
x_train, x_predict, y_train, y_predict
# x_train.tofile("/home/fonttian/Data/UCI/wine/wine_x_train.csv")

np.savetxt(fname='/home/fonttian/Data/UCI/wine/wine_x_train.csv',X = x_train,delimiter=',',fmt='%f')
np.savetxt(fname='/home/fonttian/Data/UCI/wine/wine_x_predict.csv',X = x_predict,delimiter=',',fmt='%f')
np.savetxt(fname='/home/fonttian/Data/UCI/wine/wine_y_train.csv',X = y_train,delimiter=',',fmt='%d')

train_pool = Pool(data='/home/fonttian/Data/UCI/wine/wine_x_train.csv',label='/home/fonttian/Data/UCI/wine/wine_y_train.csv',delimiter=',')
test_pool = Pool(data='/home/fonttian/Data/UCI/wine/wine_x_predict.csv',label=y_predict,delimiter=',')

#### 测试读取是否成功

In [50]:
# Initialize CatBoostClassifier
model = CatBoostRegressor(n_estimators=1000,learning_rate=1,depth=7, loss_function='RMSE',random_seed=100,early_stopping_rounds = 100)
# Fit model
model.fit(train_pool)
# Get predicted result
prediction = model.predict(test_pool)
from sklearn.metrics import mean_squared_error
print("RMSE of the example : %.3f "%np.sqrt(mean_squared_error(y_predict, prediction)))

0:	learn: 1.6035011	total: 16.5ms	remaining: 16.5s
1:	learn: 1.1386435	total: 32.5ms	remaining: 16.2s
2:	learn: 1.0246556	total: 48.2ms	remaining: 16s
3:	learn: 0.9401409	total: 70.8ms	remaining: 17.6s
4:	learn: 0.9281364	total: 83.5ms	remaining: 16.6s
5:	learn: 0.9178729	total: 92.9ms	remaining: 15.4s
6:	learn: 0.8976922	total: 102ms	remaining: 14.5s
7:	learn: 0.8785297	total: 111ms	remaining: 13.8s
8:	learn: 0.8479494	total: 120ms	remaining: 13.2s
9:	learn: 0.8449684	total: 129ms	remaining: 12.7s
10:	learn: 0.8006562	total: 138ms	remaining: 12.4s
11:	learn: 0.7995567	total: 146ms	remaining: 12s
12:	learn: 0.7905742	total: 154ms	remaining: 11.7s
13:	learn: 0.7860104	total: 163ms	remaining: 11.5s
14:	learn: 0.7853768	total: 171ms	remaining: 11.2s
15:	learn: 0.7845491	total: 180ms	remaining: 11.1s
16:	learn: 0.7842949	total: 188ms	remaining: 10.9s
17:	learn: 0.7842267	total: 193ms	remaining: 10.5s
18:	learn: 0.7839695	total: 201ms	remaining: 10.4s
19:	learn: 0.7803254	total: 210ms	remai

172:	learn: 0.5151448	total: 1.86s	remaining: 8.9s
173:	learn: 0.5151108	total: 1.87s	remaining: 8.89s
174:	learn: 0.5149886	total: 1.89s	remaining: 8.91s
175:	learn: 0.5149223	total: 1.9s	remaining: 8.9s
176:	learn: 0.5142161	total: 1.91s	remaining: 8.88s
177:	learn: 0.5140668	total: 1.92s	remaining: 8.87s
178:	learn: 0.5132127	total: 1.93s	remaining: 8.86s
179:	learn: 0.5123203	total: 1.94s	remaining: 8.85s
180:	learn: 0.5122472	total: 1.96s	remaining: 8.85s
181:	learn: 0.5122255	total: 1.97s	remaining: 8.84s
182:	learn: 0.5120721	total: 1.98s	remaining: 8.82s
183:	learn: 0.5119186	total: 1.98s	remaining: 8.8s
184:	learn: 0.5118437	total: 1.99s	remaining: 8.78s
185:	learn: 0.5117751	total: 2s	remaining: 8.77s
186:	learn: 0.5070812	total: 2.01s	remaining: 8.75s
187:	learn: 0.4984564	total: 2.02s	remaining: 8.74s
188:	learn: 0.4980487	total: 2.03s	remaining: 8.72s
189:	learn: 0.4980227	total: 2.04s	remaining: 8.68s
190:	learn: 0.4978461	total: 2.04s	remaining: 8.66s
191:	learn: 0.49766

341:	learn: 0.4175834	total: 3.46s	remaining: 6.65s
342:	learn: 0.4169598	total: 3.47s	remaining: 6.64s
343:	learn: 0.4168547	total: 3.48s	remaining: 6.64s
344:	learn: 0.4168041	total: 3.49s	remaining: 6.62s
345:	learn: 0.4167862	total: 3.5s	remaining: 6.61s
346:	learn: 0.4166840	total: 3.51s	remaining: 6.6s
347:	learn: 0.4166093	total: 3.52s	remaining: 6.59s
348:	learn: 0.4165762	total: 3.52s	remaining: 6.58s
349:	learn: 0.4165198	total: 3.54s	remaining: 6.57s
350:	learn: 0.4165049	total: 3.54s	remaining: 6.55s
351:	learn: 0.4164443	total: 3.55s	remaining: 6.54s
352:	learn: 0.4164280	total: 3.57s	remaining: 6.54s
353:	learn: 0.4157669	total: 3.58s	remaining: 6.54s
354:	learn: 0.4157343	total: 3.6s	remaining: 6.55s
355:	learn: 0.4156496	total: 3.63s	remaining: 6.56s
356:	learn: 0.4155204	total: 3.65s	remaining: 6.57s
357:	learn: 0.4154792	total: 3.66s	remaining: 6.57s
358:	learn: 0.4154501	total: 3.68s	remaining: 6.57s
359:	learn: 0.4154486	total: 3.69s	remaining: 6.55s
360:	learn: 0.4

518:	learn: 0.3852311	total: 5.26s	remaining: 4.87s
519:	learn: 0.3852127	total: 5.27s	remaining: 4.86s
520:	learn: 0.3851469	total: 5.28s	remaining: 4.85s
521:	learn: 0.3851442	total: 5.29s	remaining: 4.84s
522:	learn: 0.3850058	total: 5.3s	remaining: 4.83s
523:	learn: 0.3849916	total: 5.32s	remaining: 4.83s
524:	learn: 0.3845553	total: 5.33s	remaining: 4.82s
525:	learn: 0.3845398	total: 5.33s	remaining: 4.81s
526:	learn: 0.3845333	total: 5.34s	remaining: 4.8s
527:	learn: 0.3845260	total: 5.35s	remaining: 4.79s
528:	learn: 0.3844771	total: 5.36s	remaining: 4.78s
529:	learn: 0.3825659	total: 5.37s	remaining: 4.76s
530:	learn: 0.3824659	total: 5.38s	remaining: 4.75s
531:	learn: 0.3822850	total: 5.39s	remaining: 4.74s
532:	learn: 0.3822767	total: 5.4s	remaining: 4.73s
533:	learn: 0.3822607	total: 5.41s	remaining: 4.72s
534:	learn: 0.3822128	total: 5.42s	remaining: 4.71s
535:	learn: 0.3821962	total: 5.43s	remaining: 4.7s
536:	learn: 0.3821465	total: 5.44s	remaining: 4.69s
537:	learn: 0.38

680:	learn: 0.3632980	total: 6.86s	remaining: 3.21s
681:	learn: 0.3632970	total: 6.87s	remaining: 3.2s
682:	learn: 0.3632561	total: 6.88s	remaining: 3.19s
683:	learn: 0.3631015	total: 6.89s	remaining: 3.18s
684:	learn: 0.3630973	total: 6.9s	remaining: 3.17s
685:	learn: 0.3630910	total: 6.91s	remaining: 3.16s
686:	learn: 0.3630717	total: 6.92s	remaining: 3.15s
687:	learn: 0.3630694	total: 6.92s	remaining: 3.14s
688:	learn: 0.3630575	total: 6.93s	remaining: 3.13s
689:	learn: 0.3630556	total: 6.94s	remaining: 3.12s
690:	learn: 0.3630449	total: 6.95s	remaining: 3.1s
691:	learn: 0.3630429	total: 6.96s	remaining: 3.1s
692:	learn: 0.3630375	total: 6.96s	remaining: 3.08s
693:	learn: 0.3630175	total: 6.97s	remaining: 3.08s
694:	learn: 0.3628605	total: 6.98s	remaining: 3.06s
695:	learn: 0.3626187	total: 6.99s	remaining: 3.05s
696:	learn: 0.3625858	total: 7s	remaining: 3.04s
697:	learn: 0.3625010	total: 7.01s	remaining: 3.03s
698:	learn: 0.3624135	total: 7.02s	remaining: 3.02s
699:	learn: 0.36217

847:	learn: 0.3516335	total: 8.86s	remaining: 1.59s
848:	learn: 0.3516074	total: 8.87s	remaining: 1.58s
849:	learn: 0.3516023	total: 8.89s	remaining: 1.57s
850:	learn: 0.3515845	total: 8.9s	remaining: 1.56s
851:	learn: 0.3501023	total: 8.91s	remaining: 1.55s
852:	learn: 0.3498745	total: 8.92s	remaining: 1.54s
853:	learn: 0.3498471	total: 8.93s	remaining: 1.53s
854:	learn: 0.3498430	total: 8.95s	remaining: 1.52s
855:	learn: 0.3498423	total: 8.96s	remaining: 1.51s
856:	learn: 0.3498410	total: 8.97s	remaining: 1.5s
857:	learn: 0.3497715	total: 8.98s	remaining: 1.49s
858:	learn: 0.3497617	total: 8.99s	remaining: 1.48s
859:	learn: 0.3496560	total: 9s	remaining: 1.47s
860:	learn: 0.3495808	total: 9.01s	remaining: 1.46s
861:	learn: 0.3494838	total: 9.02s	remaining: 1.44s
862:	learn: 0.3494746	total: 9.03s	remaining: 1.43s
863:	learn: 0.3491999	total: 9.04s	remaining: 1.42s
864:	learn: 0.3491741	total: 9.06s	remaining: 1.41s
865:	learn: 0.3491718	total: 9.08s	remaining: 1.4s
866:	learn: 0.3491

可能是`numpy。savetxt()`方法存储过程中导致数据变化的原因，新的测试用例结果显示RMSE大幅度上升。不过因为本处只是语法展示，而在真是的使用过程中是不会产生这个问题的，所以这个问题此处可以忽略。